<a href="https://colab.research.google.com/github/schradern/datathon_2026/blob/main/00_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites

Change PROJECT_ID to your own project ID.

In [1]:
PROJECT_ID = "" #@param {type:"string"}

# sets default dataset for AmsterdamUMCdb
DATASET_PROJECT_ID = 'amsterdamumcdb' #@param {type:"string"}
DATASET_ID = 'van_gogh_2026_datathon' #@param {type:"string"}
LOCATION = 'eu' #@param {type:"string"}

In [2]:
import os
from google.colab import auth
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
%load_ext google.colab.data_table
from google.colab.data_table import DataTable

# change default limits:
DataTable.max_columns = 50
DataTable.max_rows = 30000

In [4]:
from google.cloud.bigquery import magics
from google.cloud import bigquery

# sets the default query job configuration
def_config = bigquery.job.QueryJobConfig(default_dataset=DATASET_PROJECT_ID + "." + DATASET_ID)
magics.context.default_query_job_config = def_config

In [5]:
import pandas as pd
import pandas_gbq

config_gbq = {'query':
          {'defaultDataset': {
              "datasetId": DATASET_ID,
              "projectId": DATASET_PROJECT_ID
              },
           'location': LOCATION}
           }

src_config = f"{DATASET_PROJECT_ID}.{DATASET_ID}"

def q(sql: str):
  return pandas_gbq.read_gbq(sql, configuration=config_gbq, use_bqstorage_api=True)

## Exploration

### concept identification

In [6]:
sql_concept_regex = r"""
-- StandardSQL
DECLARE pattern STRING DEFAULT r"{pattern}";

WITH matches AS (
  SELECT concept_id, concept_code, concept_name, domain_id, vocabulary_id
  FROM `amsterdamumcdb.van_gogh_2026_datathon.concept`
  WHERE REGEXP_CONTAINS(LOWER(concept_name), pattern)
     OR REGEXP_CONTAINS(LOWER(concept_code), pattern)
),
counts AS (
  SELECT
    "measurement" AS table_domain,
    m.measurement_concept_id AS concept_id,
    COUNT(*) AS n_rows,
    COUNT(DISTINCT m.person_id) AS n_patients
  FROM `amsterdamumcdb.van_gogh_2026_datathon.measurement` m
  WHERE m.measurement_concept_id IN (SELECT concept_id FROM matches)
  GROUP BY 2

  UNION ALL
  SELECT
    "observation" AS table_domain,
    o.observation_concept_id AS concept_id,
    COUNT(*) AS n_rows,
    COUNT(DISTINCT o.person_id) AS n_patients
  FROM `amsterdamumcdb.van_gogh_2026_datathon.observation` o
  WHERE o.observation_concept_id IN (SELECT concept_id FROM matches)
  GROUP BY 2

  UNION ALL
  SELECT
    "drug_exposure" AS table_domain,
    d.drug_concept_id AS concept_id,
    COUNT(*) AS n_rows,
    COUNT(DISTINCT d.person_id) AS n_patients
  FROM `amsterdamumcdb.van_gogh_2026_datathon.drug_exposure` d
  WHERE d.drug_concept_id IN (SELECT concept_id FROM matches)
  GROUP BY 2

  UNION ALL
  SELECT
    "device_exposure" AS table_domain,
    x.device_concept_id AS concept_id,
    COUNT(*) AS n_rows,
    COUNT(DISTINCT x.person_id) AS n_patients
  FROM `amsterdamumcdb.van_gogh_2026_datathon.device_exposure` x
  WHERE x.device_concept_id IN (SELECT concept_id FROM matches)
  GROUP BY 2
)
SELECT
  c.table_domain,
  m.domain_id AS concept_domain,
  m.concept_id,
  m.concept_code,
  m.concept_name,
  m.vocabulary_id,
  c.n_rows,
  c.n_patients
FROM counts c
JOIN matches m ON m.concept_id = c.concept_id
WHERE c.n_rows > 0
ORDER BY c.n_rows DESC
"""

def regex_concepts_with_counts(pattern: str):
    if not pattern:
        raise ValueError("pattern must be non-empty (empty regex matches everything).")
    pattern = pattern.replace('"', r'\"')
    return q(sql_concept_regex.format(pattern=pattern))

In [7]:
#ventilation_concepts = regex_concepts_with_counts(r"(vent|ventilat|peep|fio2|cpap|bipap|oxygen|trach|endotrache)")
#ventilation_concepts = regex_concepts_with_counts(r"(bloed|venous|arterial)")
#ventilation_concepts = regex_concepts_with_counts(r"(systolic|diastolic|spo2|nasal|face)")
#ventilation_concepts

### domain specific methods

In [8]:
DOMAIN_SPEC = {
    "measurement": dict(
        concept_col="measurement_concept_id",
        date_col="measurement_date",
        datetime_col="measurement_datetime",
        numeric_col="value_as_number",
        unit_col="unit_source_value",
        main_cat_col="value_source_value",
    ),
    "observation": dict(
        concept_col="observation_concept_id",
        date_col="observation_date",
        datetime_col="observation_datetime",
        numeric_col="value_as_number",
        unit_col="unit_source_value",
        main_cat_col="value_source_value",
    ),
    "drug_exposure": dict(
        concept_col="drug_concept_id",
        date_col="drug_exposure_start_date",
        datetime_col=None,
        numeric_col="quantity",
        unit_col="dose_unit_source_value",
        main_cat_col="drug_source_value",
    ),
    "condition_occurrence": dict(
        concept_col="condition_concept_id",
        date_col="condition_start_date",
        datetime_col=None,
        numeric_col=None,
        unit_col=None,
        main_cat_col="condition_source_value",
    ),
    "device_exposure": dict(
        concept_col="device_concept_id",
        date_col="device_exposure_start_date",
        datetime_col=None,
        numeric_col=None,
        unit_col=None,
        main_cat_col="device_source_value",
    ),
    "visit_occurrence": dict(
        concept_col="visit_concept_id",
        date_col="visit_start_date",
        datetime_col=None,
        numeric_col=None,
        unit_col=None,
        main_cat_col="visit_source_value",
    ),
}

In [9]:
def _normalize_concept_ids(concept_ids):
    label_map = None

    if isinstance(concept_ids, int):
        return [int(concept_ids)], None

    if isinstance(concept_ids, dict):
        label_map = {int(k): str(v) for k, v in concept_ids.items()}
        return list(label_map.keys()), label_map

    if isinstance(concept_ids, list) and concept_ids and all(isinstance(x, dict) for x in concept_ids):
        label_map = {int(k): str(v) for d in concept_ids for k, v in d.items()}
        return list(label_map.keys()), label_map

    ids = [int(x) for x in list(concept_ids)]
    return ids, None


def explore_concepts_summary(
    concept_ids,
    domain: str,
    dataset: str = src_config,
    top_n: int = 15,
    max_units: int = 50,
):
    spec = DOMAIN_SPEC.get(domain)
    if spec is None:
        raise ValueError(f"Unsupported domain={domain!r}. Supported: {', '.join(DOMAIN_SPEC)}")

    ids, label_map = _normalize_concept_ids(concept_ids)
    ids_sql = "[" + ",".join(map(str, ids)) + "]"

    concept_col  = spec["concept_col"]
    date_col     = spec["date_col"]
    datetime_col = spec["datetime_col"]
    numeric_col  = spec["numeric_col"]
    unit_col     = spec["unit_col"]
    cat_col      = spec["main_cat_col"]

    numeric_expr = numeric_col if numeric_col else "CAST(NULL AS FLOAT64)"
    unit_expr    = unit_col if unit_col else "CAST(NULL AS STRING)"
    cat_expr     = cat_col if cat_col else "CAST(NULL AS STRING)"

    if numeric_col:
        numeric_agg = """
          COUNTIF(numeric_value IS NOT NULL) AS n_numeric_rows,
          MIN(numeric_value) AS min_value,
          MAX(numeric_value) AS max_value,
          AVG(numeric_value) AS mean_value,
          STDDEV(numeric_value) AS sd_value,
          APPROX_QUANTILES(numeric_value, 100) AS q,
        """
    else:
        numeric_agg = """
          0 AS n_numeric_rows,
          CAST(NULL AS FLOAT64) AS min_value,
          CAST(NULL AS FLOAT64) AS max_value,
          CAST(NULL AS FLOAT64) AS mean_value,
          CAST(NULL AS FLOAT64) AS sd_value,
          CAST(NULL AS ARRAY<FLOAT64>) AS q,
        """

    if domain in ("measurement", "observation"):
        ts_expr = (
            f"COALESCE({datetime_col}, TIMESTAMP({date_col}))"
            if datetime_col else
            f"TIMESTAMP({date_col})"
        )

        interval_cte = f"""
        , interval_stats AS (
          WITH ordered AS (
            SELECT
              {concept_col} AS concept_id,
              person_id,
              {ts_expr} AS ts
            FROM `{dataset}.{domain}`
            WHERE {concept_col} IN UNNEST({ids_sql})
          ),
          diffs AS (
            SELECT
              concept_id,
              person_id,
              TIMESTAMP_DIFF(
                ts,
                LAG(ts) OVER (PARTITION BY concept_id, person_id ORDER BY ts),
                SECOND
              ) AS diff_s
            FROM ordered
          ),
          patient_avg AS (
            SELECT
              concept_id,
              person_id,
              AVG(diff_s) AS patient_avg_diff_s,
              COUNT(diff_s) AS n_intervals
            FROM diffs
            WHERE diff_s IS NOT NULL AND diff_s > 0
            GROUP BY concept_id, person_id
          )
          SELECT
            concept_id,
            AVG(patient_avg_diff_s) / 3600.0 AS avg_patient_mean_interval_hours,
            AVG(patient_avg_diff_s) / 86400.0 AS avg_patient_mean_interval_days,
            APPROX_QUANTILES(patient_avg_diff_s, 100)[SAFE_OFFSET(50)] / 3600.0
              AS median_patient_mean_interval_hours,
            COUNT(*) AS n_patients_with_intervals,
            SUM(n_intervals) AS total_intervals
          FROM patient_avg
          GROUP BY concept_id
        )
        """
    else:
        interval_cte = """
        , interval_stats AS (
          SELECT
            a.concept_id AS concept_id,
            CAST(NULL AS FLOAT64) AS avg_patient_mean_interval_hours,
            CAST(NULL AS FLOAT64) AS avg_patient_mean_interval_days,
            CAST(NULL AS FLOAT64) AS median_patient_mean_interval_hours,
            0 AS n_patients_with_intervals,
            0 AS total_intervals
          FROM agg a
        )
        """

    summary_sql = f"""
    WITH data AS (
      SELECT
        {concept_col} AS concept_id,
        person_id,
        {numeric_expr} AS numeric_value,
        {unit_expr} AS unit_value,
        {cat_expr} AS cat_value
      FROM `{dataset}.{domain}`
      WHERE {concept_col} IN UNNEST({ids_sql})
    ),
    agg AS (
      SELECT
        concept_id,
        COUNT(*) AS n_rows,
        COUNT(DISTINCT person_id) AS n_patients,

        {numeric_agg}

        COUNTIF(unit_value IS NOT NULL AND unit_value != "") AS n_unit_rows,
        COUNT(DISTINCT NULLIF(unit_value, "")) AS n_unique_units,

        COUNTIF(cat_value IS NOT NULL AND cat_value != "") AS n_cat_rows,
        COUNT(DISTINCT NULLIF(cat_value, "")) AS n_unique_cat_values
      FROM data
      GROUP BY concept_id
    ),
    units AS (
      SELECT
        concept_id,
        ARRAY_AGG(unit_value ORDER BY n_rows DESC LIMIT {int(max_units)}) AS units
      FROM (
        SELECT concept_id, unit_value, COUNT(*) AS n_rows
        FROM data
        WHERE unit_value IS NOT NULL AND unit_value != ""
        GROUP BY concept_id, unit_value
      )
      GROUP BY concept_id
    ),
    topvals AS (
      SELECT
        concept_id,
        ARRAY_AGG(cat_value ORDER BY n_rows DESC LIMIT {int(top_n)}) AS top_values
      FROM (
        SELECT concept_id, cat_value, COUNT(*) AS n_rows
        FROM data
        WHERE cat_value IS NOT NULL AND cat_value != ""
        GROUP BY concept_id, cat_value
      )
      GROUP BY concept_id
    )
    {interval_cte}
    SELECT
      "{domain}" AS domain,
      a.concept_id,
      c.concept_code,
      c.concept_name,
      c.domain_id AS concept_domain,

      a.n_rows,
      a.n_patients,

      a.n_numeric_rows,
      a.min_value,
      a.max_value,
      a.mean_value,
      a.sd_value,
      a.q[SAFE_OFFSET(5)]  AS p05,
      a.q[SAFE_OFFSET(25)] AS p25,
      a.q[SAFE_OFFSET(50)] AS p50,
      a.q[SAFE_OFFSET(75)] AS p75,
      a.q[SAFE_OFFSET(95)] AS p95,

      a.n_unit_rows,
      a.n_unique_units,
      COALESCE(u.units, ARRAY<STRING>[]) AS units,

      a.n_cat_rows,
      a.n_unique_cat_values,
      COALESCE(t.top_values, ARRAY<STRING>[]) AS top_values,

      i.avg_patient_mean_interval_hours,
      i.avg_patient_mean_interval_days,
      i.median_patient_mean_interval_hours,
      i.n_patients_with_intervals,
      i.total_intervals
    FROM agg a
    JOIN `{dataset}.concept` c
      ON c.concept_id = a.concept_id
    LEFT JOIN units u
      ON u.concept_id = a.concept_id
    LEFT JOIN topvals t
      ON t.concept_id = a.concept_id
    LEFT JOIN interval_stats i
      ON i.concept_id = a.concept_id
    ORDER BY a.n_rows DESC
    """

    df = q(summary_sql)
    if label_map is not None:
        df = df.copy()
        df["label"] = df["concept_id"].map(label_map)
    return df

### Exploring Data

#### Ventilation

In [10]:
vent_concept_ids: dict[int, str] = {
    2000000203: "FiO2 non-invasive",  # n_pat=77, n_rows=942, units=['%'], p25=21.0, p50=21.0, p75=21.0, med_int_h=4.75
    2000000250: "Ventilation mode non-invasive",  # n_pat=19, n_rows=115, units=['cmH2O'], p25=8.0, p50=8.0, p75=10.0, med_int_h=1.1777777777777778
    2000000634: "Non-invasive ventilator leakage fraction",  # n_pat=49, n_rows=1013, units=['%'], p25=0.0, p50=0.0, p75=21.0, med_int_h=3.716666666666667

    2000000629: "Tube depth", # n_pat=16789, n_rows=93310, units=['cm'], p25=22.0, p50=23.0, p75=24.0, med_int_h=23.825
    21491186: "Endotracheal tube diameter",  # n_pat=31847, n_rows=64033, units=[], p25=7.5, p50=8.0, p75=8.5, med_int_h=384.05
    36305611: "Tracheostomy tube diameter",  # n_pat=845, n_rows=1407, units=[], p25=7.0, p50=8.0, p75=8.0, med_int_h=1295.0
    2000000173: "Tracheostomy tube type",  # n_pat=31, n_rows=32, categorical-only (no numeric summary/intervals in this extract)
    2000000232: "Tracheobronchial toilet event",  # n_pat=9493, n_rows=205773, units=[], p25=1.0, p50=1.0, p75=2.0, med_int_h=7.5

    2000000204: "FiO2 setting",  # n_pat=37457, n_rows=15506049, units=['%' 'Geen'], p25=40.0, p50=46.0, p75=63.0, med_int_h=0.9238095238095239
    2000000205: "FiO2 placeholder",  # n_pat=2654, n_rows=130575, units=['Geen' '%'], p25=22.0, p50=40.0, p75=40.0, med_int_h=2.5275507614213204
    3020716: "Inhaled oxygen concentration",  # n_pat=519, n_rows=120653, units=['%'], p25=0.55, p50=0.6, p75=0.7, med_int_h=0.016666666666666666
    3025408: "FiO2 by O2 analyzer (on ventilator)",  # n_pat=9090, n_rows=711934, units=['%'], p25=34.8, p50=40.3, p75=50.6, med_int_h=1.0185185185185184
    3005629: "Inhaled oxygen flow rate",  # n_pat=58360, n_rows=3061903, units=['L/min' 'l' 'l/min' 'Geen' 'cmH2O'], p25=2.0, p50=3.0, p75=5.0, med_int_h=1.764705882352941

    3004921: "Ventilation mode (ventilator)",  # n_pat=324, n_rows=683, categorical-only in this extract; med_int_h=1.5 (from interval stats)

    3022875: "PEEP setting (ventilator)",  # n_pat=36626, n_rows=10425042, units=['cmH2O'], p25=5.0, p50=5.0, p75=8.0, med_int_h=0.987037037037037
    3012410: "Tidal volume setting (ventilator)",  # n_pat=23524, n_rows=5818150, units=['ml'], p25=400.0, p50=450.0, p75=500.0, med_int_h=0.018079096045197748
    3045410: "Minute volume setting (ventilator)",  # n_pat=25080, n_rows=3048511, units=['l/min' 'L/min' 'bpm'], p25=5.7, p50=7.2, p75=10.1, med_int_h=1.0
    3007469: "Breath rate setting (ventilator)",  # n_pat=44840, n_rows=5711113, units=['br/min' '/min' 'bpm'], p25=12.0, p50=15.0, p75=20.0, med_int_h=1.0
    36304672: "Inspiratory time setting (ventilator)",  # n_pat=30748, n_rows=5057597, units=['s' 'sec'], p25=1.4, p50=3.4, p75=15.0, med_int_h=0.9666666666666667
    2000000220: "Inspiratory time setting (%)",  # n_pat=7, n_rows=27, units=['Geen'], p25=29.0, p50=33.0, p75=39.0, med_int_h=2.0
    3000461: "Pressure support setting (ventilator)",  # n_pat=13792, n_rows=952384, units=['cm H2O' '+PEEP'], p25=5.0, p50=8.0, p75=12.0, med_int_h=1.0874048037492683
    2000000209: "Inspiratory Pressure Above PEEP (set)",  # n_pat=29103, n_rows=3113240, units=['cmH2O' 'cm H2O' 'cmH20'], p25=10.0, p50=18.0, p75=40.0, med_int_h=1.0
    2000000211: "Inspiratory Pressure Above PEEP",  # n_pat=17354, n_rows=5102095, units=['cmH2O'], p25=6.73, p50=11.0, p75=585.0, med_int_h=0.025925925925925925
    3009065: "Inspiration/Expiration setting (I:E)",  # n_pat=12426, n_rows=882020, units=['Geen' 'cmH2O'], p25=30.0, p50=30.0, p75=30.0, med_int_h=1.0314069264069272
    40760768: "Flow trigger setting",  # n_pat=9319, n_rows=887035, units=['l/min' 'cmH2O'], p25=1.61, p50=3.0, p75=20.0, med_int_h=0.875
    36305890: "Inflation trigger pressure setting",  # n_pat=7059, n_rows=325733, units=['cm H2O' 'cmH2O'], p25=5.0, p50=5.0, p75=5.0, med_int_h=1.0072332730560578
    3011557: "Peak inspiratory flow setting",  # n_pat=720, n_rows=29078, units=['l/min' 'L/min'], p25=55.0, p50=59.0, p75=83.4, med_int_h=1.0
    36305682: "Bias flow setting",  # n_pat=34, n_rows=128, units=['L/min' 'l/min'], p25=40.0, p50=45.0, p75=50.0, med_int_h=1.0

    3043148: "Breath rate mechanical (on ventilator)",  # n_pat=16756, n_rows=1026257, units=['bpm' 'l/min'], p25=17.0, p50=22.0, p75=29.0, med_int_h=1.0
    3026892: "Breath rate spontaneous (on ventilator)",  # n_pat=14173, n_rows=834116, units=['bpm' '/min'], p25=14.0, p50=20.0, p75=27.0, med_int_h=1.0
    21490854: "Tidal volume (on ventilator)",  # n_pat=23540, n_rows=236013, units=['mL'], p25=368.46, p50=412.08, p75=455.52, med_int_h=695.5
    3017594: "Tidal volume spontaneous (on ventilator)",  # n_pat=9631, n_rows=641549, units=['mL' 'ml'], p25=0.0, p50=372.0, p75=498.0, med_int_h=1.0454545454545456
    3017878: "Tidal volume (spont+mech)/body weight (on ventilator)",  # n_pat=33, n_rows=1534, units=['ml/kg'], p25=5.32, p50=6.39, p75=7.64, med_int_h=1.3333333333333333

    3016226: "PEEP respiratory system",  # n_pat=24639, n_rows=3208912, units=['cmH2O' 'mbar'], p25=5.0, p50=5.80000018671562, p75=8.69999996888073, med_int_h=1.0
    3035822: "Intrinsic PEEP",  # n_pat=13769, n_rows=2916879, units=['cmH20' 'mbar' 'cm H2O'], p25=1.0, p50=2.0, p75=4.0, med_int_h=0.8
    42527140: "Total PEEP",  # n_pat=7933, n_rows=1667756, units=['cmH2O' 'cmH20'], p25=44.0, p50=51.0, p75=61.0, med_int_h=0.026619718309859142

    21490879: "Ventilator airway circuit leakage volume",  # n_pat=25, n_rows=179, units=[], p25=11.0, p50=22.0, p75=47.0, med_int_h=1.2083333333333333

    2000000188: "BiPAP high pressure",  # n_pat=362, n_rows=16398, units=['mbar'], p25=22.0, p50=26.0, p75=30.0, med_int_h=1.0172413793103448
    2000000191: "BiPAP low pressure",  # n_pat=358, n_rows=16267, units=['mbar'], p25=8.0, p50=10.0, p75=12.0, med_int_h=1.0333333333333334
    2000000190: "BiPAP inspiratory pressure set",  # n_pat=24, n_rows=392, units=['cmH2O'], p25=10.0, p50=14.999994709724119, p75=18.000000622385397, med_int_h=1.75

    2000000238: "Vent mechanics peak airway pressure",  # n_pat=37236, n_rows=5796940, units=['cmH2O'], p25=14.0, p50=18.0, p75=22.799998630752125, med_int_h=1.0
    36306157: "Airway pressure max (on ventilator)",  # n_pat=12630, n_rows=3848310, units=['mbar' 'cmH2O'], p25=15.0, p50=24.0, p75=34.0, med_int_h=0.020420660276890323
    2000000235: "Vent mechanics compliance respiratory",  # n_pat=87, n_rows=343, units=[], p25=23.13, p50=35.0, p75=57.5, med_int_h=80.0
    2000000236: "Vent mechanics mechanical power",  # n_pat=45, n_rows=71, units=[], p25=18.77, p50=26.42, p75=33.0, med_int_h=55.75
    2000000239: "Vent mechanics Paw total PEEP",  # n_pat=89, n_rows=190, units=[], p25=10.0, p50=12.0, p75=15.0, med_int_h=74.81666666666666
    2000000244: "Vent mechanics transpulmonary pressure exp",  # n_pat=79, n_rows=164, units=[], p25=-1.3, p50=0.0, p75=3.0, med_int_h=87.5
    2000000245: "Vent mechanics transpulmonary pressure insp",  # n_pat=79, n_rows=163, units=[], p25=8.0, p50=11.0, p75=16.0, med_int_h=87.5
    2000000246: "Vent mechanics volume end inspiration",  # n_pat=87, n_rows=182, units=[], p25=8.71, p50=12.04, p75=15.64, med_int_h=80.0

    2000000241: "Vent mechanics PES exp occlusion",  # n_pat=102, n_rows=352, units=[], p25=8.0, p50=14.0, p75=40.0, med_int_h=58.785714285714285
    2000000243: "Vent mechanics PES insp occlusion",  # n_pat=79, n_rows=163, units=[], p25=11.0, p50=14.0, p75=17.0, med_int_h=87.5
    2000000240: "Vent mechanics PES exp dyn",  # n_pat=36, n_rows=48, units=[], p25=-8.0, p50=1.0, p75=10.0, med_int_h=120.0
    2000000242: "Vent mechanics PES insp dyn",  # n_pat=36, n_rows=47, units=[], p25=4.0, p50=8.0, p75=13.0, med_int_h=91.5

    2000000210: "Inspiratory Pressure Above PEEP (measured)",  # n_pat=30, n_rows=310, units=['cmH20'], p25=12.0, p50=15.0, p75=18.0, med_int_h=6.833333333333333
}


#vent_exploration = explore_concepts_summary(vent_concept_ids, domain="measurement", top_n=5)
#vent_exploration

#### BGA / laboratory

In [11]:

bga_lab_concept_ids: dict[int, str] = {
    3027801: "PaO2 (arterial pO2)",  # n_pat=35379, n_rows=1446041, units=['kPa'], p25=10.4, p50=13.4, p75=23.5, med_int_h=3.87047619047619
    3027315: "pO2 in blood (unspecified specimen)",  # n_pat=18442, n_rows=654290, units=['kPa'], p25=10.4, p50=19.7, p75=75.8, med_int_h=3.56969696969697
    3016502: "SaO2 (arterial O2 saturation)",  # n_pat=42554, n_rows=1284380, units=['Geen' '%'], p25=0.9599999785423279, p50=0.9800000190734863, p75=53.4, med_int_h=3.953333333333333
    3011367: "O2 saturation calculated from pO2",  # n_pat=680, n_rows=3007, units=[], p25=0.65, p50=0.73, p75=0.83, med_int_h=12.0
    3044904: "Oxygen content in blood",  # n_pat=284, n_rows=2454, units=['mL/dL'], p25=7.0, p50=8.0, p75=10.0, med_int_h=9.744318181818183

    3024928: "Venous O2 saturation",  # n_pat=7528, n_rows=311516, units=['%'], p25=74.0, p50=79.0, p75=85.0, med_int_h=4.745833333333334
    3024354: "Venous pO2",  # n_pat=7001, n_rows=14798, units=['kPa'], p25=4.5, p50=5.3, p75=6.5, med_int_h=15.016666666666667

    3018465: "Mixed venous O2 saturation",  # n_pat=1550, n_rows=5098, units=[], p25=0.57, p50=0.64, p75=0.71, med_int_h=5.5777777777777775
    3038071: "Mixed venous pO2",  # n_pat=1364, n_rows=2123, units=['kPa'], p25=4.4, p50=5.1, p75=5.9, med_int_h=14.166666666666666
    1616373: "Central venous O2 saturation",  # n_pat=127, n_rows=1545, units=['%' 'Geen'], p25=58.0, p50=66.0, p75=72.0, med_int_h=1.6

    3021901: "Capillary O2 saturation",  # n_pat=52, n_rows=72, units=[], p25=0.83, p50=0.91, p75=0.95, med_int_h=52.56666666666667
    3028626: "Capillary pO2",  # n_pat=19, n_rows=25, units=['kPa'], p25=8.0, p50=9.4, p75=11.4, med_int_h=11.675

    3027946: "PaCO2 (arterial pCO2)",  # n_pat=35381, n_rows=1436824, units=['kPa'], p25=4.7, p50=5.3, p75=6.0, med_int_h=3.8785714285714286
    3021447: "Venous pCO2",  # n_pat=6657, n_rows=13925, units=['kPa'], p25=5.0, p50=5.7, p75=6.4, med_int_h=15.930392156862744
    3021513: "Mixed venous pCO2",  # n_pat=1111, n_rows=1654, units=['kPa'], p25=5.3, p50=5.9, p75=6.5, med_int_h=13.783333333333333
    1616989: "Central venous pCO2",  # n_pat=83, n_rows=120, units=['kPa'], p25=5.3, p50=5.9, p75=6.7, med_int_h=5.633333333333334

    3019977: "Arterial pH",  # n_pat=23793, n_rows=1076559, units=[], p25=7.34, p50=7.4, p75=7.45, med_int_h=3.9145833333333333
    3012544: "Venous pH",  # n_pat=12568, n_rows=35117, units=[], p25=7.3, p50=7.36, p75=7.41, med_int_h=15.416666666666666
    3012388: "Mixed venous pH",  # n_pat=2669, n_rows=6801, units=[], p25=7.3, p50=7.34, p75=7.38, med_int_h=6.061111111111111

    3008152: "Arterial bicarbonate (HCO3-)",  # n_pat=24631, n_rows=1083644, units=['mmol/L' 'mEq/l' 'mmol/l'], p25=21.2, p50=24.0, p75=27.0, med_int_h=3.90462962962963
    3027273: "Venous bicarbonate (HCO3-)",  # n_pat=12392, n_rows=34102, units=['mmol/L' 'mEq/l'], p25=20.8, p50=23.9, p75=26.9, med_int_h=15.291666666666666
    3025817: "Mixed venous bicarbonate (HCO3-)",  # n_pat=2650, n_rows=6772, units=['mmol/L'], p25=21.7, p50=23.8, p75=25.6, med_int_h=6.061111111111111

    3003396: "Arterial base excess",  # n_pat=27234, n_rows=1066121, units=['mmol/L' 'mEq/L'], p25=-2.9, p50=0.0, p75=2.8, med_int_h=4.03
    3002032: "Venous base excess",  # n_pat=12386, n_rows=33973, units=['mmol/L' 'mEq/L'], p25=-4.9, p50=-1.4, p75=1.9, med_int_h=15.323333333333334
    3023001: "Mixed venous base excess",  # n_pat=2625, n_rows=6660, units=['mmol/L'], p25=-4.0, p50=-1.7, p75=0.3, med_int_h=6.083333333333333

    43533606: "Arterial anion gap (calc, 4 ions)",  # n_pat=8977, n_rows=305624, units=['mmol/L'], p25=5.0, p50=6.0, p75=9.0, med_int_h=3.5819444444444444
    43533603: "Venous anion gap (calc, 4 ions)",  # n_pat=5543, n_rows=17888, units=['mmol/L'], p25=5.0, p50=7.0, p75=10.0, med_int_h=12.125
    43533604: "Mixed venous anion gap (calc, 4 ions)",  # n_pat=2083, n_rows=6173, units=['mmol/L'], p25=5.0, p50=6.0, p75=8.0, med_int_h=6.466666666666667

    3018405: "Arterial lactate",  # n_pat=23347, n_rows=518880, units=['mmol/L'], p25=1.0, p50=1.3, p75=1.9, med_int_h=4.276190476190474
    3008037: "Venous lactate",  # n_pat=11169, n_rows=28451, units=['mmol/L'], p25=1.1, p50=1.6, p75=2.6, med_int_h=14.755555555555556
    3005949: "Mixed venous lactate",  # n_pat=2244, n_rows=5953, units=['mmol/L'], p25=1.2, p50=1.7, p75=2.6, med_int_h=5.866666666666666

    3044242: "Arterial glucose",  # n_pat=23684, n_rows=659329, units=['mmol/L'], p25=6.6, p50=7.8, p75=9.3, med_int_h=3.806481481481481
    3038515: "Venous glucose",  # n_pat=12193, n_rows=35965, units=['mmol/L'], p25=6.1, p50=7.5, p75=9.3, med_int_h=13.54
    2000000141: "Mixed venous glucose",  # n_pat=2371, n_rows=6300, units=['mmol/L'], p25=7.3, p50=8.7, p75=10.0, med_int_h=6.0

    3043409: "Arterial potassium",  # n_pat=23698, n_rows=658084, units=['mmol/L'], p25=3.8, p50=4.1, p75=4.5, med_int_h=3.8333333333333335
    3041354: "Venous potassium",  # n_pat=12212, n_rows=36111, units=['mmol/L'], p25=3.8, p50=4.1, p75=4.6, med_int_h=13.65
    3031219: "Mixed venous potassium",  # n_pat=2357, n_rows=6274, units=['mmol/L'], p25=4.0, p50=4.3, p75=4.7, med_int_h=5.933333333333334

    3043706: "Arterial sodium",  # n_pat=23662, n_rows=587237, units=['mmol/L'], p25=135.0, p50=139.0, p75=143.0, med_int_h=4.264367816091954
    3041473: "Venous sodium",  # n_pat=11752, n_rows=32845, units=['mmol/L'], p25=134.0, p50=137.0, p75=141.0, med_int_h=14.433333333333334
    3031579: "Mixed venous sodium",  # n_pat=2303, n_rows=6146, units=['mmol/L'], p25=136.0, p50=138.0, p75=140.0, med_int_h=5.875

    3031248: "Arterial chloride",  # n_pat=23576, n_rows=516190, units=['mmol/L'], p25=103.0, p50=107.0, p75=111.0, med_int_h=4.658333333333333
    3035285: "Venous chloride",  # n_pat=11202, n_rows=29756, units=['mmol/L'], p25=102.0, p50=106.0, p75=110.0, med_int_h=15.65
    2000000137: "Mixed venous chloride",  # n_pat=2192, n_rows=5932, units=['mmol/L'], p25=106.0, p50=108.0, p75=111.0, med_int_h=5.766666666666667

    3044331: "Arterial ionized calcium",  # n_pat=14993, n_rows=208724, units=['mmol/L'], p25=1.08, p50=1.14, p75=1.19, med_int_h=5.927777777777778
    3033705: "Venous ionized calcium",  # n_pat=5655, n_rows=12140, units=['mmol/L'], p25=1.05, p50=1.13, p75=1.19, med_int_h=18.95
    3032271: "Mixed venous ionized calcium",  # n_pat=676, n_rows=901, units=['mmol/L'], p25=1.07, p50=1.13, p75=1.19, med_int_h=10.916666666666666

    21490721: "Arterial hemoglobin",  # n_pat=23672, n_rows=745726, units=['mmol/L' '%'], p25=5.0, p50=5.9, p75=7.1, med_int_h=3.9205234159779607
    1002216: "Venous hemoglobin",  # n_pat=15261, n_rows=51011, units=['mmol/L' '%'], p25=5.4, p50=6.3, p75=7.4, med_int_h=9.466666666666667
    2000000145: "Mixed venous hemoglobin",  # n_pat=3342, n_rows=9413, units=['mmol/L' '%'], p25=5.8, p50=6.7, p75=7.9, med_int_h=6.9

    3023230: "Arterial hematocrit (calculated)",  # n_pat=8983, n_rows=314259, units=['L/L'], p25=0.26, p50=0.3, p75=0.35, med_int_h=3.4999999999999996
    3034976: "Venous hematocrit (calculated)",  # n_pat=22542, n_rows=509534, units=['Geen' 'L/L'], p25=0.2800000011920929, p50=0.3100000023841858, p75=0.36000001430511475, med_int_h=3.3361111111111112
    3034037: "Mixed venous hematocrit (calculated)",  # n_pat=1539, n_rows=5082, units=['L/L'], p25=0.29, p50=0.33, p75=0.37, med_int_h=5.583333333333333

    42869594: "Arterial oxyhemoglobin fraction",  # n_pat=8986, n_rows=314650, units=[], p25=0.92, p50=0.94, p75=0.96, med_int_h=3.558333333333333
    42869591: "Venous oxyhemoglobin fraction",  # n_pat=15564, n_rows=52202, units=['%'], p25=0.77, p50=56.4, p75=70.9, med_int_h=9.874444444444444
    42869592: "Mixed venous oxyhemoglobin fraction",  # n_pat=3351, n_rows=8339, units=['%'], p25=0.6, p50=0.72, p75=61.5, med_int_h=6.9

    3006217: "Arterial methemoglobin fraction",  # n_pat=23605, n_rows=617333, units=['%'], p25=0.2, p50=0.3, p75=0.7, med_int_h=4.159803921568628
    3024889: "Venous methemoglobin fraction",  # n_pat=15167, n_rows=50354, units=['%'], p25=0.3, p50=0.4, p75=0.7, med_int_h=9.563333333333333
    3031282: "Mixed venous methemoglobin fraction",  # n_pat=3312, n_rows=8247, units=['%'], p25=0.4, p50=0.6, p75=0.8, med_int_h=6.9

    3028653: "Arterial carboxyhemoglobin fraction",  # n_pat=23596, n_rows=615634, units=['%'], p25=0.6, p50=1.0, p75=1.4, med_int_h=4.172435897435897
    3034030: "Mixed venous carboxyhemoglobin fraction",  # n_pat=3302, n_rows=8233, units=['%'], p25=0.7, p50=1.0, p75=1.3, med_int_h=6.9
}

#lab_exploration = explore_concepts_summary(bga_lab_concept_ids, domain="measurement", top_n=5)
#lab_exploration

#### Vitals

In [12]:
vital_concept_ids: dict[int, str] = {
    40762499: "SpO2 (pulse oximetry)",  # n_pat=25131, n_rows=15126034, units=['%'], p25=96.5, p50=99.0, p75=100.0, med_int_h=2.7893401015228427
    21490670: "Heart rate (intra-arterial line, invasive)",  # n_pat=45, n_rows=4330, units=['/min'], p25=76.0, p50=89.0, p75=102.0, med_int_h=1.0333333333333332

    21490853: "Invasive systolic blood pressure (A-line)",  # n_pat=47720, n_rows=8905484, units=['mmHg'], p25=98.0, p50=114.0, p75=134.0, med_int_h=1.0145985401459854
    21490851: "Invasive diastolic blood pressure (A-line)",  # n_pat=36773, n_rows=7175006, units=['mmHg'], p25=51.99999862659608, p50=59.0, p75=67.0, med_int_h=1.009345794392523

    21490675: "Central venous pressure (CVP)",  # n_pat=28596, n_rows=3419053, units=['mmHg'], p25=3.0, p50=7.0, p75=11.999996795390857, med_int_h=1.5
    3000333: "CVP mean",  # n_pat=15401, n_rows=1127030, units=['mmHg'], p25=6.0, p50=9.0, p75=14.0, med_int_h=1.0454545454545456

    21492239: "Noninvasive systolic blood pressure",  # n_pat=11031, n_rows=125187, units=['mmHg'], p25=104.00002655247576, p50=118.99998523590787, p75=139.00000583696666, med_int_h=2.6333333333333333
    21492240: "Noninvasive diastolic blood pressure",  # n_pat=9727, n_rows=121663, units=['mmHg'], p25=56.9999982832451, p50=66.99999759654314, p75=76.99998226019939, med_int_h=2.316666666666667

    3005606: "Pulmonary artery systolic blood pressure (PA line)",  # n_pat=7577, n_rows=190399, units=['mmHg'], p25=24.9999982832451, p50=32.0, p75=40.00000091560261, med_int_h=1.0
    3017188: "Pulmonary artery diastolic blood pressure (PA line)",  # n_pat=7473, n_rows=190023, units=['mmHg'], p25=11.999996795390857, p50=16.0, p75=21.000001487854245, med_int_h=1.0

    21490906: "Nasal temperature",  # n_pat=8921, n_rows=1804984, units=["C'"], p25=95.6, p50=97.0, p75=97.9, med_int_h=0.016666666666666666

    2000000108: "PiCCO global end-diastolic volume (GEDV)",  # n_pat=874, n_rows=25602, units=['ml'], p25=1103.0, p50=1380.0, p75=1702.0, med_int_h=1.8041666666666667
    2000000109: "PiCCO global end-diastolic volume index (GEDVI)",  # n_pat=1071, n_rows=24708, units=['ml/m2'], p25=619.0, p50=748.0, p75=899.0, med_int_h=3.3636363636363638

    21490712: "Cardiac index (left ventricular)",  # n_pat=2580, n_rows=102844, units=['L/min' 'l/min/m2' '(L/min)/BSA'], p25=2.01, p50=2.5, p75=3.2, med_int_h=2.1126893939393945
    21492228: "Cardiac output continuous (left ventricular)",  # n_pat=2195, n_rows=95308, units=['L/Min' 'l/min'], p25=4.0, p50=5.0, p75=6.4, med_int_h=1.4782608695652175
    3005555: "Cardiac output (left ventricular)",  # n_pat=8923, n_rows=61892, units=['l/min'], p25=4.0, p50=5.0, p75=6.199999809265137, med_int_h=4.0
    3036518: "Stroke volume (left ventricular)",  # n_pat=2294, n_rows=51129, units=['ml'], p25=51.310001373291016, p50=65.8, p75=83.0, med_int_h=3.151282051282051
    21490880: "Stroke volume index (left ventricular)",  # n_pat=556, n_rows=22104, units=[], p25=27.0, p50=33.0, p75=40.0, med_int_h=1.6666666666666667
    2000000117: "Stroke volume variation (left ventricular)",  # n_pat=450, n_rows=18401, units=['%' 'Geen'], p25=10.0, p50=16.0, p75=22.0, med_int_h=1.5205882352941176
    3002309: "Cardiac output by indicator dilution",  # n_pat=130, n_rows=2164, units=['l/min'], p25=4.900000095367432, p50=6.0, p75=7.46999979019165, med_int_h=4.715277777777777

    3016369: "Premature ventricular contractions",  # n_pat=681, n_rows=855, categorical-only in this extract; med_int_h=3.0
    21490598: "Oxygen consumption (VO2)",  # n_pat=6464, n_rows=1029359, units=[], p25=159.0, p50=204.0, p75=252.0, med_int_h=0.037730061349693256
}

#vital_exploration = explore_concepts_summary(vital_concept_ids, domain="measurement", top_n=5)
#vital_exploration

### Devices

In [13]:
device_concept_ids: dict[int: str] = {
    2000000432: "Pacemaker ventricle output amplitude",  # n_pat=3054, n_rows=10358, units=['mA'], p25=2.0, p50=3.0, p75=6.0, med_int_h=24.0
    2000000421: "Pacemaker ventricle sensing",  # n_pat=1384, n_rows=2840, units=[], p25=3.0, p50=6.0, p75=10.0, med_int_h=19.433333333333334
    2000000393: "Pacemaker ventricular sensitivity threshold",  # n_pat=1789, n_rows=3266, units=['mV'], p25=1.0, p50=2.0, p75=3.0, med_int_h=20.65

    2000000099: "ECMO arterial flow",  # n_pat=96, n_rows=9048, units=['L/min'], p25=0.0, p50=0.0, p75=0.0, med_int_h=1.010416666666667
    2000000100: "ECMO venous flow",  # n_pat=1, n_rows=123, units=['l/min'], p25=2.62, p50=2.8, p75=3.1, med_int_h=1.2377049180327866
    2000000102: "ECMO arterial pressure",  # n_pat=304, n_rows=29060, units=['mmHg'], p25=-19.0, p50=145.0, p75=173.0, med_int_h=1.1142857142857143
    2000000103: "ECMO venous pressure",  # n_pat=255, n_rows=23811, units=['mmHg'], p25=-41.0, p50=-22.0, p75=-4.0, med_int_h=1.1139896373057
    2000000419: "ECMO arterial temperature",  # n_pat=2457, n_rows=451125, units=[], p25=82.0, p50=90.7, p75=95.0, med_int_h=0.016666666666666666
    2000000400: "ECMO venous temperature",  # n_pat=183, n_rows=12969, units=[], p25=90.7, p50=97.16, p75=98.24, med_int_h=1.0113636363636365
    2000000101: "ECMO FiO2",  # n_pat=225, n_rows=21917, units=['%' '.' 'Geen'], p25=50.0, p50=61.0, p75=76.0, med_int_h=1.0833333333333333

    2000000614: "Renal replacement therapy venous pressure",  # n_pat=1766, n_rows=176725, units=['mmHg'], p25=60.0, p50=75.0, p75=101.0, med_int_h=1.1796610169491528
    2000000623: "RRT arterial pressure",  # n_pat=1717, n_rows=171527, units=['mmHg'], p25=-57.0, p50=-40.0, p75=-25.0, med_int_h=1.1797752808988768
}

#device_exploration = explore_concepts_summary(device_concept_ids, domain="measurement", top_n=5)
#device_exploration